<img src="https://raw.githubusercontent.com/LuisVelasc/Imagenes/main/RNConcreto.PNG" alt="drawing" width="850">

En este notebook se gardan las notas tomadas del curso de edX **Deep Learning Fundamentals with Keras**, impartido por **Alex Aklson** de IBM Developer Skills Network. En el ejercicio se realiza una red neuronal para predecir la resistencia a compresión del concreto. El concreto es un material heterogeneo compuesto por muchos materiales como áridos, agua, cemento, aditivos, adiciones, etc. Esta complejidad en su composición dificulta el crear modelos predictivos para estimar las propiedades mecánicas del concreto, sin embargo, al ser este material fundamental para la industria de la construcción, el contar con herraminetas que nos permitan asegurar su calidad es un tema de gran importancia.

## Librerías y base de datos

Empezamos cargando las librerías necesarias para el ejemplo.

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

from keras.models import Sequential
from keras.layers import Dense

Ahora cargamos la base de datos con la que alimentaremos a nuestra red. Como se puede observar, la base de datos contiene la cantindad de cemento, humo de silice, ceniza volante, agua, superplastificante, agregado grueso y fino, la edad en días a la que se probó la probeta y la resistencia obtenida en MPa.

In [2]:
concrete_data = pd.read_csv('C:/Users/fable/Documents/Jupyter Notebook/RNConcreto/concrete_data_E&T.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


Ahora generamos los estadísticos descriptivos de nuestra base de datos. Contamos con 899 datos y no hay datos faltantes. En este caso, el parámetro importante es la resistencia a compresión, ese es el valor que nos interesa predecir. Se observa que de media, la resistencia a compresión de la muestra presenta un valor de 36.4 MPa, estándo este valor entre el intervalo de 2 MPa y 82.6 MPa. El que el 25% de las probetas hayan presentado una resistencia a compresión por debajo de 25 MPa hace desconfiar un poco de la calidad de esta base de datos. 

In [3]:
concrete_data.describe().T

,count,mean,std,min,25%,50%,75%,max
Cement,899.0,290.725139,104.589986,102.00,212.00,277.10,362.600,540.0
Blast Furnace Slag,899.0,68.574305,85.834843,0.00,0.00,19.00,132.400,359.4
Fly Ash,899.0,48.937709,61.868140,0.00,0.00,0.00,114.500,200.0
Water,899.0,180.722358,21.773827,121.80,163.40,185.00,192.000,247.0
Superplasticizer,899.0,5.805117,6.174551,0.00,0.00,5.70,10.150,32.2
Coarse Aggregate,899.0,981.070634,75.544945,801.00,935.40,972.60,1043.600,1145.0
Fine Aggregate,899.0,776.193548,81.421827,594.00,739.15,780.10,828.000,992.6
Age,899.0,48.235818,67.234092,1.00,7.00,28.00,56.000,365.0
Strength,899.0,36.400623,17.202553,2.33,23.52,35.23,47.795,82.6


## Preparación de la base de datos

El primer paso es dividir la base de datos entre predictores y objetivos. El primer grupo se utilizará como valores de entrada para nuestra red mientras que el segundo corresponde al valor que nosotros queremos predecir, en este caso sería la resistencia a compresión.

In [4]:
# Se genera vector con nombre de columnas
concrete_data_columns = concrete_data.columns

# Los predictores son aquellas datos que no están en la columnas "Strength"
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']]
target = concrete_data['Strength']

Revisamos si se ha realizado bien la separación de los datos.

In [5]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [6]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Para obtener un mejor desempeño de nuestra red, necesitamos estandarizar nuestros datos. Esto se realiza en las siguientes líneas de código. Observese que los predictores estandarizados se guardan en otra variable.

In [7]:
escalador=preprocessing.StandardScaler()
dsE=escalador.fit_transform(predictors)

predictors_norm=pd.DataFrame(dsE,columns=predictors.columns)

predictors_norm.describe().T

,count,mean,std,min,25%,50%,75%,max
Cement,899.0,-9.039858e-17,1.000557,-1.805433,-0.753121,-0.130344,0.687588,2.384680
Blast Furnace Slag,899.0,-7.113331e-17,1.000557,-0.799355,-0.799355,-0.577876,0.744001,3.390087
Fly Ash,899.0,-9.484441e-17,1.000557,-0.791440,-0.791440,-0.791440,1.060300,2.443041
Water,899.0,3.161480e-16,1.000557,-2.707616,-0.796002,0.196567,0.518233,3.045608
Superplasticizer,899.0,0.000000e+00,1.000557,-0.940692,-0.940692,-0.017034,0.704068,4.277166
Coarse Aggregate,899.0,1.343629e-16,1.000557,-2.384949,-0.604886,-0.112189,0.828172,2.171166
Fine Aggregate,899.0,-6.243924e-16,1.000557,-2.238896,-0.455212,0.048005,0.636626,2.659323
Age,899.0,3.951851e-17,1.000557,-0.702949,-0.613658,-0.301143,0.115544,4.713985


Necesitaremos el número de columnas de nuestras variables predictoras para construir nuestra red neuronal. Después de este paso podemos empezar a construir nuestra red neuronal.

In [8]:
n_cols = predictors_norm.shape[1]
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.384680,-0.799355,-0.79144,-0.860335,-0.535579,0.780491,-1.231234,-0.301143
1,2.384680,-0.799355,-0.79144,-0.860335,-0.535579,0.979159,-1.231234,-0.301143
2,0.399638,0.861734,-0.79144,2.172515,-0.940692,-0.649917,-2.238896,3.300225
3,0.399638,0.861734,-0.79144,2.172515,-0.940692,-0.649917,-2.238896,4.713985
4,-0.881312,0.744001,-0.79144,0.518233,-0.940692,-0.035371,0.605905,4.639577


## Construcción de la red neuronal

Construir la red neuronal es muy sencillo si se utilizan las funciones brindadas por **keras**. En este caso se utilizan cuatro capas: una de entrada , dos ocultas con 50 neuronas y una de salida. Como función de activación se utiliza la función **relu**. Como algoritmo de optimización de pesos se utiliza el algoritmo **adam** mientras que el error se mide utilizando la **media de la sumatoria de los errores al cuadrado**.

In [9]:
def regression_model():
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # Compilación del modelo
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

El modelo se guarda en el objeto **model**.

In [10]:
model = regression_model()

## Entrenamiento de la red neuronal

Para entrenar la red se utilizan los valores de entrada estandarizados, se realizan 500 iteraciones y se apartan el 30% de los datos para la validación de la red.

In [11]:
model.fit(predictors_norm, target, validation_split=0.3, epochs=500, verbose=2)

Epoch 1/500
20/20 - 1s - loss: 1775.5326 - val_loss: 996.6049
Epoch 2/500
20/20 - 0s - loss: 1657.0323 - val_loss: 882.9353
Epoch 3/500
20/20 - 0s - loss: 1462.6104 - val_loss: 717.4285
Epoch 4/500
20/20 - 0s - loss: 1171.3937 - val_loss: 514.0877
Epoch 5/500
20/20 - 0s - loss: 806.1758 - val_loss: 323.3802
Epoch 6/500
20/20 - 0s - loss: 472.2364 - val_loss: 206.4454
Epoch 7/500
20/20 - 0s - loss: 288.3836 - val_loss: 169.7230
Epoch 8/500
20/20 - 0s - loss: 235.5811 - val_loss: 161.6728
Epoch 9/500
20/20 - 0s - loss: 218.7333 - val_loss: 154.3379
Epoch 10/500
20/20 - 0s - loss: 207.6287 - val_loss: 151.4446
Epoch 11/500
20/20 - 0s - loss: 199.6566 - val_loss: 148.2883
Epoch 12/500
20/20 - 0s - loss: 193.5406 - val_loss: 145.6081
Epoch 13/500
20/20 - 0s - loss: 188.5463 - val_loss: 143.3323
Epoch 14/500
20/20 - 0s - loss: 183.7487 - val_loss: 142.6925
Epoch 15/500
20/20 - 0s - loss: 178.4079 - val_loss: 140.4546
Epoch 16/500
20/20 - 0s - loss: 174.5843 - val_loss: 138.8800
Epoch 17/500


Epoch 135/500
20/20 - 0s - loss: 30.7028 - val_loss: 71.6753
Epoch 136/500
20/20 - 0s - loss: 30.4704 - val_loss: 72.2056
Epoch 137/500
20/20 - 0s - loss: 29.7643 - val_loss: 71.5662
Epoch 138/500
20/20 - 0s - loss: 29.8776 - val_loss: 70.6565
Epoch 139/500
20/20 - 0s - loss: 29.4941 - val_loss: 73.2215
Epoch 140/500
20/20 - 0s - loss: 29.0509 - val_loss: 72.2243
Epoch 141/500
20/20 - 0s - loss: 28.6263 - val_loss: 67.5485
Epoch 142/500
20/20 - 0s - loss: 28.3267 - val_loss: 72.2721
Epoch 143/500
20/20 - 0s - loss: 28.0230 - val_loss: 71.4688
Epoch 144/500
20/20 - 0s - loss: 27.6943 - val_loss: 73.1891
Epoch 145/500
20/20 - 0s - loss: 27.6628 - val_loss: 70.4389
Epoch 146/500
20/20 - 0s - loss: 27.3217 - val_loss: 67.4828
Epoch 147/500
20/20 - 0s - loss: 27.0426 - val_loss: 69.4705
Epoch 148/500
20/20 - 0s - loss: 26.9866 - val_loss: 71.5878
Epoch 149/500
20/20 - 0s - loss: 26.6621 - val_loss: 68.6258
Epoch 150/500
20/20 - 0s - loss: 26.4925 - val_loss: 68.8452
Epoch 151/500
20/20 - 0s

Epoch 270/500
20/20 - 0s - loss: 15.6360 - val_loss: 61.6605
Epoch 271/500
20/20 - 0s - loss: 15.3098 - val_loss: 60.0802
Epoch 272/500
20/20 - 0s - loss: 15.2968 - val_loss: 60.3010
Epoch 273/500
20/20 - 0s - loss: 15.2749 - val_loss: 58.7209
Epoch 274/500
20/20 - 0s - loss: 15.3124 - val_loss: 59.7182
Epoch 275/500
20/20 - 0s - loss: 15.3288 - val_loss: 60.3241
Epoch 276/500
20/20 - 0s - loss: 15.3054 - val_loss: 61.4445
Epoch 277/500
20/20 - 0s - loss: 15.6344 - val_loss: 59.9002
Epoch 278/500
20/20 - 0s - loss: 15.7301 - val_loss: 60.9719
Epoch 279/500
20/20 - 0s - loss: 15.5384 - val_loss: 59.3611
Epoch 280/500
20/20 - 0s - loss: 14.8890 - val_loss: 60.5313
Epoch 281/500
20/20 - 0s - loss: 14.9010 - val_loss: 62.1717
Epoch 282/500
20/20 - 0s - loss: 14.8584 - val_loss: 60.4880
Epoch 283/500
20/20 - 0s - loss: 14.7469 - val_loss: 60.8417
Epoch 284/500
20/20 - 0s - loss: 14.5897 - val_loss: 58.7024
Epoch 285/500
20/20 - 0s - loss: 14.7512 - val_loss: 60.7595
Epoch 286/500
20/20 - 0s

Epoch 405/500
20/20 - 0s - loss: 11.9384 - val_loss: 62.7547
Epoch 406/500
20/20 - 0s - loss: 12.1996 - val_loss: 63.2879
Epoch 407/500
20/20 - 0s - loss: 12.3286 - val_loss: 67.9521
Epoch 408/500
20/20 - 0s - loss: 12.9629 - val_loss: 66.0525
Epoch 409/500
20/20 - 0s - loss: 12.0995 - val_loss: 64.6166
Epoch 410/500
20/20 - 0s - loss: 12.8901 - val_loss: 65.3374
Epoch 411/500
20/20 - 0s - loss: 12.1510 - val_loss: 64.6739
Epoch 412/500
20/20 - 0s - loss: 12.0748 - val_loss: 63.6143
Epoch 413/500
20/20 - 0s - loss: 12.2184 - val_loss: 63.9606
Epoch 414/500
20/20 - 0s - loss: 12.0123 - val_loss: 63.6372
Epoch 415/500
20/20 - 0s - loss: 12.8472 - val_loss: 63.7954
Epoch 416/500
20/20 - 0s - loss: 12.2219 - val_loss: 65.4635
Epoch 417/500
20/20 - 0s - loss: 11.8380 - val_loss: 64.6411
Epoch 418/500
20/20 - 0s - loss: 12.0248 - val_loss: 64.7372
Epoch 419/500
20/20 - 0s - loss: 11.8072 - val_loss: 64.1061
Epoch 420/500
20/20 - 0s - loss: 11.8805 - val_loss: 66.5206
Epoch 421/500
20/20 - 0s

## Evaluación de la red neuronal

Observamos la valores que predice nuestra red neuronal para nuestros predictores estandarizados y los guardamos en la variable **Expec**. La variable es de tipo numpy.

In [12]:
Expec=model.predict(predictors_norm)

Ahora observamos el error entre las resistencias predichas y las registradas en la base de datos. Para facilitar la interpretación de los resultados se convierte el resultado en un data frame y se obtienen sus estadísticos descriptivos. Se observa que el **error medio fue del 12%**, estándo el 75% de los arrores con un valor menor al 15%.

In [13]:
P=(concrete_data[["Strength"]]).to_numpy()
Er=abs(Expec-P)/P*100
Er=pd.DataFrame(Er)
Er.describe()

,0
count,899.000000
mean,13.975975
std,29.501217
min,0.000592
25%,2.802464
50%,6.656195
75%,14.688164
max,499.043032
